# CS838 Project Stage3

Yuying Chen
Maria Kamenetsky

In [45]:
#import modules
import csv
import py_entitymatching as em
import pandas as pd
import numpy as np
import timeit
import sys

We first set our goals: we want to create a matcher that achieves precsion of at least 90% and recall as high as possible. In our team, we then went through and explored the idea and defined what we would consider to be a match and not a match. We elaborate below

## Read input tables

Since the original two movie dataset that we selected don't have too many overlaps, we changed one of them to another movie dataset. The first dataset is from kaggle.com. The second dataset is from movielens.org.

We added an id column for both datasets.

In [3]:
A = em.read_csv_metadata('../DATA/movie1_stage3.csv', key='id',encoding='mac_roman')
B = em.read_csv_metadata('../DATA/movie2_stage3.csv', key='movieId',encoding='mac_roman')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [4]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A x B: ' + str(len(A)*len(B)))

Number of tuples in A: 5043
Number of tuples in B: 27278
Number of tuples in A x B: 137562954


In [5]:
A.head() 

,id,title,genres,director_name,plot_keywords,actor_3_facebook_likes,cast_total_facebook_likes,content_rating,budget,duration,...,actor_2_facebook_likes,imdb_score,movie_facebook_likes,language,actor_1_facebook_likes,facenumber_in_poster,actor_1_name,num_user_for_reviews,aspect_ratio,num_critic_for_reviews
0,1,avatar,action|adventure|fantasy|sci-fi,james cameron,avatar|future|marine|native|paraplegic,855,4834,pg-13,237000000,178,...,936,7.9,33000,english,1000,0,cch pounder,3054,1.78,723
1,2,pirates of the caribbean: at world's end,action|adventure|fantasy,gore verbinski,goddess|marriage ceremony|marriage proposal|pirate|singapore,1000,48350,pg-13,300000000,169,...,5000,7.1,0,english,40000,0,johnny depp,1238,2.35,302
2,3,spectre,action|adventure|thriller,sam mendes,bomb|espionage|sequel|spy|terrorist,161,11700,pg-13,245000000,148,...,393,6.8,85000,english,11000,1,christoph waltz,994,2.35,602
3,4,the dark knight rises,action|thriller,christopher nolan,deception|imprisonment|lawlessness|police officer|terrorist plot,23000,106759,pg-13,250000000,164,...,23000,8.5,164000,english,27000,0,tom hardy,2701,2.35,813
4,5,star wars: episode vii - the force awakens,documentary,doug walker,.,.,143,.,.,.,...,12,7.1,0,.,131,0,doug walker,.,.,.


In [6]:
A.columns.values

array(['id', 'title', 'genres', 'director_name', 'plot_keywords',
       'actor_3_facebook_likes', 'cast_total_facebook_likes',
       'content_rating', 'budget', 'duration', 'country', 'color', 'year',
       'num_voted_users', 'director_facebook_likes', 'gross',
       'actor_2_name', 'actor_3_name', 'actor_2_facebook_likes',
       'imdb_score', 'movie_facebook_likes', 'language',
       'actor_1_facebook_likes', 'facenumber_in_poster', 'actor_1_name',
       'num_user_for_reviews', 'aspect_ratio', 'num_critic_for_reviews'], dtype=object)

In [7]:
B.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [8]:
B.columns.values

array(['movieId', 'title', 'genres', 'year'], dtype=object)

In [9]:
em.get_key(A), em.get_key(B)

('id', 'movieId')

Since the tables are very large let's downsample:

In [10]:
A1, B1 = em.down_sample(A,B, 200,1, show_progress=True)
len(A1), len(B1)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


(167, 200)

## Block Tables To Get Candidate Set

Block on attribute 'year' using an overlap blocker since the same movie must have the same year. The output attributes of table A is limited to id, title, year and genres to match the information provided by table B.

In [59]:
# Create overlap blocker
ob = em.OverlapBlocker()

start = time.clock()
C = ob.block_tables(A, B, 'year', 'year',
                   l_output_attrs=['id', 'title', 'year', 'genres'],
                   r_output_attrs=['movieId', 'title','year', 'genres'],
                   overlap_size=1, show_progress=True)
elapsed = (time.clock - start)
print("Overlap blocking step: ", elapsed)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:15


Overlap blocking step:  0.43176930748960274


In [60]:
em.to_csv_metadata(C, '../DATA/candidate_pairs.csv') 

File already exists at ../DATA/candidate_pairs.csv; Overwriting it
Metadata file already exists at ../DATA/candidate_pairs.metadata. Overwriting it


True

In [12]:
len(C)

3288743

## Match tuple pairs in candidate set

### Import Pre-Determined Golden Data

We randomly sampled 500 tuple pairs for labeling. There are 201 matches and 299 mismatches. **gold.csv** is our gold dataset of perfect matches that we have determined prior to this. 

In [13]:
G = em.read_csv_metadata('../DATA/gold.csv', 
                         key='_id',
                            ltable=A, rtable=B, 
                        fk_ltable='ltable_id', fk_rtable='rtable_movieId',
                         encoding='mac_roman')
len(G)

Metadata file is not present in the given path; proceeding to read the csv file.


500

In [14]:
G.head(10)

,_id,ltable_id,rtable_movieId,ltable_movie_title,ltable_title_year,ltable_genres,rtable_title,rtable_year,rtable_genres,label
0,2433,894,64716,seven pounds,2008,drama|romance,Seven Pounds,2008,Drama,1
1,4545,2842,4015,"dude, where's my car?",2000,comedy|mystery,"Dude, Where's My Car?",2000,Comedy|Sci-Fi,1
2,6907,3697,104908,insidious: chapter 2,2013,fantasy|horror|thriller,Insidious: Chapter 2,2013,Horror|Thriller,1
3,24168,4384,2891,"happy, texas",1999,comedy|crime|romance,"Happy, Texas",1999,Comedy,1
4,31352,1844,7980,a bridge too far,1977,drama|history|war,"Bridge Too Far, A",1977,Action|Drama|War,1
5,35987,874,2676,instinct,1999,drama|thriller,Instinct,1999,Drama|Thriller,1
6,37526,2582,1599,steel,1997,action|crime|sci-fi,Steel,1997,Action,1
7,50124,1212,1408,the last of the mohicans,1992,action|adventure|drama|romance|war,"Last of the Mohicans, The",1992,Action|Romance|War|Western,1
8,57296,2987,66200,two lovers,2008,drama|romance,Two Lovers,2008,Drama|Romance,1
9,64913,2952,2841,stir of echoes,1999,horror|mystery|thriller,Stir of Echoes,1999,Horror|Mystery|Thriller,1


We note here, that based on what we saw in the creation of the golden data, we establish a *correct* match as a perfect match on year and title, where title is lower case. For example, we would not consider tuple *(insidious, 2013)* and tuple *(insidious: chapter 2, 2013)* to be a match. We do so because we cannot reliably say that the sequal refers to the original. 

### Splitting Labeled Data into Development and Evaluation Set

We next want to split our data into a development set, where we will iteratively train and test various classifiers on the data. The evaluation set will be our final hold out set which we will test last and only once, so that we avoid overfitting. 

Let *I* be the development set and *J* be the evaluation set. We split the data into a 70-30 split for development-evaluation.

In [15]:
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

In [16]:
#write these to .csvs in DATA folder 
I.to_csv('../DATA/setI.csv')
J.to_csv('../DATA/setJ.csv')

**Package note**: the above split_train_test did not work if I imported G using pandas read_csv, even with encoding. I had to import it using
    ```em.read_csv_metadata()```, which is a little confusing to first-time user since it's a csv, but not necessarily metadata.
    Is there another function that would import metadata and do something with it separately? Or a way to interface with ```pd.read_csv()``` so that  you can read in a csv, clean it and work with it, and then reimport with ```em.read_csv_metadata()``` somehow?

### Selecting Best Learning-Based Matcher

We first go through and set our various machine-learning matchers that we will cycle through:

In [17]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

In exploring the **py_entitymatching**, we first start with using the automatically generated features.

In [61]:
#get the auto features

start = time.clock()
feature_table = em.get_features_for_matching(A,B)
elapsed = (time.clock() - start)
print("Getting features step: ", elapsed)

Getting features step:  0.5886104051711527


In [46]:
feature_table.feature_name

0        title_title_jac_qgm_3_qgm_3
1    title_title_cos_dlm_dc0_dlm_dc0
2    title_title_jac_dlm_dc0_dlm_dc0
3                    title_title_mel
4               title_title_lev_dist
5                title_title_lev_sim
6                    title_title_nmw
7                     title_title_sw
Name: feature_name, dtype: object

**Package Note**: The ```get_features_for_matching()``` doesn't work if A and B do not have any common features. Which totally makes sense, but the error message was not clear. Also, it would be nice if for features that are quantitative (for example, length of token), you could also get average length of token, median, sd, based on some sort of blocking scheme. Would this help increase P/R?

In [19]:
#convert to feature vectors
start = time.clock()
H = em.extract_feature_vecs(I,
                           feature_table=feature_table,
                           attrs_after='label',
                           show_progress=True)
elapsed = (time.clock() - start)
print("Matching step - recall: ", elapsed)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


In [20]:
H.head()

,_id,ltable_id,rtable_movieId,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_jac_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,title_title_nmw,title_title_sw,label
141,640561,3382,69253,0.368421,0.333333,0.2,0.878788,2,0.818182,9.0,9.0,1
383,1475,2654,93492,0.000000,0.000000,0.0,0.533333,9,0.100000,-4.0,2.0,0
135,620734,4840,2127,0.314286,0.000000,0.0,0.878788,4,0.818182,18.0,18.0,1
493,2413,117,64716,0.000000,0.000000,0.0,0.484127,10,0.166667,-3.0,1.0,0
122,573655,716,112556,0.294118,0.000000,0.0,0.851852,2,0.777778,7.0,7.0,1


### Selecting best ML Matcher

Now we want to select the best matcher. We use k-fold cross validation. We will use 10 folds here and use precision, recall, and F1 to select the best matcher.



In [58]:
#select best ML matcher based on cross-validation
##PRECISION
start = time.clock()
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H,
                          exclude_attrs=['_id', 'ltable_id','rtable_movieId', 'label'],
                          k = 10,
                          target_attr='label', metric='precision', random_state=0)
elapsed = (time.clock() - start)
print("Matching step - precision: ", elapsed)

Matching step - precision:  0.43176930748960274


In [54]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x00000223C2636278>,10,1.0,1.0,1.0,1.0,0.928571,1.0,1.0,1.0,1.0,1.0,0.992857
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x00000223C2636908>,10,1.0,1.0,1.0,1.0,0.928571,1.0,1.0,1.0,1.0,1.0,0.992857
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x00000223C2636860>,10,1.0,1.0,1.0,1.0,0.857143,1.0,1.0,1.0,1.0,1.0,0.985714
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x00000223C26367F0>,10,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.000000
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x00000223C2636780>,10,1.0,1.0,1.0,1.0,0.923077,1.0,1.0,1.0,1.0,1.0,0.992308
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x00000223C2636198>,10,1.0,1.0,1.0,1.0,0.923077,1.0,1.0,1.0,1.0,1.0,0.992308


Based on average precision across the 10 folds, we see that linear regression performs the best, follwed by decision tree and random forest performing equally well. 

**Package Note**: I really like the way these results are displayed - very clear and useful!

In [62]:
#select best ML matcher based on cross-validation
##RECALL
start = time.clock()
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H,
                          exclude_attrs=['_id', 'ltable_id','rtable_movieId', 'label'],
                          k = 10,
                          target_attr='label', metric='recall', random_state=0)
elapsed = (time.clock() - start)
print("Matching step - recall: ", elapsed)

Matching step - recall:  0.39812798846895703


In [57]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x00000223C2636278>,10,0.933333,1.000000,1.0000,0.928571,1.000000,1.0,0.933333,1.000000,1.000000,1.0,0.979524
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x00000223C2636908>,10,0.933333,1.000000,1.0000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,0.993333
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x00000223C2636860>,10,0.933333,0.923077,0.9375,0.928571,0.923077,1.0,1.000000,0.916667,0.857143,1.0,0.941937
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x00000223C26367F0>,10,0.933333,1.000000,1.0000,0.857143,0.846154,1.0,0.933333,1.000000,1.000000,1.0,0.956996
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x00000223C2636780>,10,0.933333,1.000000,1.0000,0.928571,0.923077,1.0,1.000000,1.000000,1.000000,1.0,0.978498
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x00000223C2636198>,10,0.933333,1.000000,1.0000,0.928571,0.923077,1.0,1.000000,1.000000,1.000000,1.0,0.978498


Based on the average recall across the 10 folds, we find that that the logistic regression performs the best, followed by Naive Bayes and than random forest/decision tree. 

For the purposes of our project, the goal is to create a matcher that achieves

**Package note**: Is there a way to extract recall at a K, which the user can specify? For example, of the top 100 tuples that are declared matches, how many of these are true positives.

#### Debug

Before proceeding, let's explore what is being misclassified:

In [23]:
C.head()

,_id,ltable_id,rtable_movieId,ltable_title,ltable_year,ltable_genres,rtable_title,rtable_year,rtable_genres
0,0,1026,1,jade,1995,crime|drama|thriller,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
1,1,2053,1,bad boys,1995,action|comedy|crime|drama|thriller,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
2,2,2822,1,clueless,1995,comedy|romance,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
3,3,4662,1,welcome to the dollhouse,1995,comedy|drama,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
4,4,3593,1,the usual suspects,1995,crime|drama|mystery|thriller,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy


In [24]:
dbg = em.debug_blocker(C, A, B, output_size = 5)

In [25]:
dbg

,_id,similarity,ltable_id,rtable_movieId,ltable_title,ltable_genres,ltable_year,rtable_title,rtable_genres,rtable_year
0,0,0.750000,1231,57854,the count of monte cristo,action|adventure|drama|romance|thriller,2002,The Count of Monte Cristo,Action|Adventure|Drama|Romance|Thriller,1934
1,1,0.750000,832,92938,ghost rider: spirit of vengeance,action|fantasy|thriller,2011,Ghost Rider: Spirit of Vengeance,Action|Fantasy|Thriller,2012
2,2,0.750000,3189,93512,"jeff, who lives at home",comedy|drama,2011,"Jeff, Who Lives at Home",Comedy|Drama,2012
3,3,0.727273,2489,2338,i know what you did last summer,horror|mystery|thriller,1997,I Still Know What You Did Last Summer,Horror|Mystery|Thriller,1998
4,4,0.714286,4443,118862,closer to the moon,comedy|drama,2014,Closer to the Moon,Comedy|Drama,2013


Based on the observations that are beign misclassified, it looks like there are two kinds of errors:

- 1) a movie gets remade and there is an old and new version of the movie
- 2) there appears to be a consistent error between the two datasets where in a subset of cases, there is a match on movie title, but year is one year off (in an inconsistent direction). This comes from how the data was originally classified in the datasets. Without going back to the source, we have no way of knowing the rules by which a movie's year was established (perhaps the difference is year it was filmed versus year when it was released).

Since we already have high precision and recall and have gained insights into the error, we leave further investigation of this error to future work. 

### Evaluating the Matching Output

First, we want to convert the feature vectors and create a new set _L_ based on the test set _J_, which will have the feature vectors from a feature table

In [63]:
start = time.clock()
L = em.extract_feature_vecs(J, feature_table=feature_table,
                           attrs_after='label',
                           show_progress=True)
elapsed = (time.clock() - start)
print("Extract features on J: ", elapsed)

0%                          100%
[##############################] | ETA: 00:00:00

Extract features on J:  0.847306780073211



Total time elapsed: 00:00:00


Next, we will trian the matched based on the feature vectors from the development set. Based on the results of the development set, we want to find a matcher that achieves precision of at least 90% and recall as high possible. Based on the nature of our project (matching movie-year across two data sources), we want to minimize the number of false negatives (minimize type 2 error). Therefore, we select the logistic regression as our matcher.

In [64]:
#Train using feature vectors from, but now on test set J
start = time.clock()
lg.fit(table=L,
      exclude_attrs=['_id', 'ltable_id', 'rtable_movieId', 'label'],
      target_attr='label')
elapsed = (time.clock() - start)
print("LG on Test Set: ", elapsed)

LG on Test Set:  0.0033999656388914445


Based on the above, we now want to predict:

In [28]:
L.head()


,_id,ltable_id,rtable_movieId,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_jac_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,title_title_nmw,title_title_sw,label
90,457561,4831,6881,0.478261,0.333333,0.2,0.911111,2,0.866667,13.0,13.0,1
254,581,4992,39183,0.000000,0.000000,0.0,0.419103,17,0.105263,0.0,3.0,0
283,771,1834,31856,0.000000,0.000000,0.0,0.481624,35,0.125000,-22.0,2.0,0
445,2013,2378,101715,0.000000,0.000000,0.0,0.351852,8,0.111111,-2.0,1.0,0
461,2169,1651,5957,0.000000,0.000000,0.0,0.384722,14,0.125000,-4.0,1.0,0


In [29]:
predictions = lg.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_movieId', 'label'], 
              append=True, target_attr='predicted', inplace=False)

### Results

In [31]:
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 100.0% (55/55)
Recall : 98.21% (55/56)
F1 : 99.1%
False positives : 0 (out of 55 positive predictions)
False negatives : 1 (out of 95 negative predictions)


In [38]:
predictions[(predictions['label'] != predictions['predicted'])]

,_id,ltable_id,rtable_movieId,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_jac_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,title_title_nmw,title_title_sw,label,predicted
142,645229,1601,47,0.074074,0.0,0.0,0.583333,16,0.2,-11.0,4.0,1,0


In [43]:
C.loc[(C['ltable_id'] == 1601) & (C['rtable_movieId'] == 47)]

,_id,ltable_id,rtable_movieId,ltable_title,ltable_year,ltable_genres,rtable_title,rtable_year,rtable_genres
3255,3255,1601,47,se7en,1995,crime|drama|mystery|thriller,Seven (a.k.a. Se7en),1995,Mystery|Thriller


**Package note**: this summary statistics output is AWESOME.

From the above test set, we can see that we have minimized false negatives (type 2 error) and still achieved precision of over 90%. Our results are very good - this was likely because our dataset had some cleaning in it from the original source. 

We are able to achieve very high recall on our test set by using the logistic regression matcher and using overlap blocking.

We again explore our mis-classified observation and find that it is misclassified due to two titles in *rtable_title* and use of "a.k.a."